In [1]:
import sys
print(sys.executable)


c:\Users\adhik\OneDrive\Desktop\job_recommendation_system\venv\Scripts\python.exe


In [2]:
import pandas as pd
import numpy as np
from rank_bm25 import BM25Okapi


In [3]:
df = pd.read_csv("../backend/models/jobs_with_features.csv")
print(df.shape)
df.head(3)


(986, 18)


,jobid,job_title,description_text,benefits,job_type,location,salary_formatted,company_name,company_rating,company_reviews_count,country,url,apply_link,job_title_lc,company_name_lc,location_lc,combined_text,description_short
0,d46480488e3aa20c,FitLab Email Marketing Manager,Description: \n Who Is FitLab: FitLab is a p...,"[""401(k)"",""Dental insurance"",""Disability insur...",Full-time,California,"$90,000 - $105,000 a year",FitLab INC,4.5,4,US,https://www.indeed.com/viewjob?jk=d46480488e3a...,https://www.indeed.com/viewjob?jk=d46480488e3a...,fitlab email marketing manager,fitlab inc,california,fitlab email marketing manager description who...,Description: \n Who Is FitLab: FitLab is a p...
1,ba9bf4e04cfd9cdb,Customer Service Representative - Work From Home,The Customer Associate is the first point of c...,"[""401(k) matching"",""Continuing education credi...",Permanent,"Sugar Land, TX 77479",$28 - $50 an hour,Buccal Up Dental,5.0,4,US,https://www.indeed.com/viewjob?jk=ba9bf4e04cfd...,https://www.indeed.com/viewjob?jk=ba9bf4e04cfd...,customer service representative - work from home,buccal up dental,"sugar land, tx 77479",customer service representative work from home...,The Customer Associate is the first point of c...
2,9e4031d06dd42d32,Information Technology Support Specialist,About Peraton Peraton is a next-generation na...,"[""Dental insurance"",""Disability insurance"",""He...",Full-time,United States,"$66,000 - $106,000 a year",Peraton,3.1,297,US,https://www.indeed.com/viewjob?jk=9e4031d06dd4...,https://www.indeed.com/viewjob?jk=9e4031d06dd4...,information technology support specialist,peraton,united states,information technology support specialist abou...,About Peraton Peraton is a next-generation na...


In [4]:
def tokenize(text):
    return text.split()

corpus = df["combined_text"].astype(str).tolist()
tokenized_corpus = [tokenize(doc) for doc in corpus]

print("Documents:", len(tokenized_corpus))
print("Sample tokens:", tokenized_corpus[0][:15])


Documents: 986
Sample tokens: ['fitlab', 'email', 'marketing', 'manager', 'description', 'who', 'is', 'fitlab', 'fitlab', 'is', 'a', 'performance', 'lifestyle', 'company', 'that']


In [5]:
bm25 = BM25Okapi(tokenized_corpus)
print("BM25 model initialized")


BM25 model initialized


In [6]:
def recommend_bm25(
    profile_text,
    top_n=10,
    location=None,
    job_type=None
):
    query_tokens = tokenize(profile_text.lower())
    scores = bm25.get_scores(query_tokens)

    results = df.copy()
    results["score"] = scores

    if location and location.lower() != "any":
        results = results[results["location"].str.contains(location, case=False, na=False)]

    if job_type and job_type.lower() != "any":
        results = results[results["job_type"].str.contains(job_type, case=False, na=False)]

    results = results.sort_values("score", ascending=False).head(top_n)

    show_cols = [c for c in [
        "job_title", "company_name", "location", "job_type", "salary_formatted", "score"
    ] if c in results.columns]

    return results[show_cols]


In [7]:
recommend_bm25("frontend developer react internship", top_n=5)


,job_title,company_name,location,job_type,salary_formatted,score
585,Java Developer Level 2,Kforce,"Cincinnati, OH 45202",Contract,$67 - $72 an hour,13.682177
356,Lead Fullstack Engineer / Java / React / WFH F...,Motion Recruitment,"Lake Forest, IL 60045",Full-time,"$150,000 - $170,000 a year",10.830277
487,Senior Technical Product Manager - Cloud Infra...,U.S. Bank National Association,"San Francisco, CA 94111",Full-time,"$145,180 - $187,880 a year",7.767723
484,"Principal Product Manager, Technical, AWS Deve...","Amazon Web Services, Inc.","San Francisco, CA",Full-time,"$163,000 - $281,800 a year",7.579444
507,"Senior Product Manager, Technical, AWS Develop...","Amazon Web Services, Inc.","San Francisco, CA",Full-time,"$137,000 - $236,800 a year",7.477968


In [8]:
recommend_bm25("data analyst python sql", top_n=5)


,job_title,company_name,location,job_type,salary_formatted,score
631,Data Analyst,DAP Health,"Palm Springs, CA 92262",Full-time,$32.49 - $33.97 an hour,12.293410
922,Quantitative Analyst - C13 - NEW YORK,Citi,"New York, NY",Full-time,"$175,000 - $250,000 a year",11.523285
220,Healthcare Analyst,Healthfirst,"100 Church St, New York, NY 10007",Full-time,"$67,200 - $97,815 a year",11.159533
884,OSAM - Quantitative Risk Analyst,Franklin Templeton Investments,"New York, NY",Full-time,"$122,000 - $145,000 a year",10.851401
189,"VP, Data Analyst (Hybrid)",Citi,"New York, NY",Full-time,"$129,840 - $194,360 a year",10.568702


In [9]:
import pickle
import os

os.makedirs("../backend/models", exist_ok=True)

with open("../backend/models/bm25_model.pkl", "wb") as f:
    pickle.dump(bm25, f)

print("BM25 model saved")


BM25 model saved
